In [ ]:
import numpy as np
import os
import matplotlib.pyplot as plt
import sys
from PIL import Image
import pickle
import h5py
import cv2
from skimage.transform import resize

sys.path.append(os.path.dirname(os.path.abspath('')))
from utils.directory import DATA_DIR, DATASET_DIR
import utils.prepare_data as pd
import utils.error as error
import utils.visualize as visual
import utils.convert_xyz_uvd as xyzuvd
# jupyter nbconvert --to html evaluation.ipynb

In [ ]:
# epoch_pose = 100
epoch_pose = 150
exp_pose = 'pose_net_aug'
from_pred_smap = True

# epoch = 265
epoch = 340
exp = 'base_lift_net'

data_split = 'train'
save_prefix = 'train_fpha'
keys_cache_file = os.path.join(DATA_DIR, save_prefix + '_keys_cache.p')
train_keys = pickle.load(open(keys_cache_file, "rb"))

dataroot_xyz_gt_saved = os.path.join(DATA_DIR, save_prefix + '_xyz_gt.lmdb')

train_xyz_gt = pd.read_all_lmdb_from_file(train_keys, dataroot_xyz_gt_saved, np.float32, (21, 3))
train_xyz_gt = np.asarray(train_xyz_gt)
train_uvd_gt = xyzuvd.xyz2uvd_color(train_xyz_gt)

In [ ]:
dataroot_xyz_gt_canon = os.path.join(DATA_DIR, save_prefix + '_xyz_gt_canon.lmdb')
dataroot_rot_mat = os.path.join(DATA_DIR, save_prefix + '_rot_mat.lmdb')
train_xyz_gt_canon = pd.read_all_lmdb_from_file(train_keys, dataroot_xyz_gt_canon, np.float32, (21, 3))
train_rot_mat = pd.read_all_lmdb_from_file(train_keys, dataroot_rot_mat, np.float32, (3, 3))

if from_pred_smap:
    pred_file = os.path.join(DATA_DIR, exp, 'xyz_canon_%s_%s_smap_%s_%s.txt' %(epoch, data_split, epoch_pose, exp_pose))
else:
    pred_file = os.path.join(DATA_DIR, exp, 'xyz_canon_%s_%s.txt' %(epoch, data_split))
train_pred_xyz_canon = pd.get_pred_xyz_canon(pred_file, save_prefix)
if from_pred_smap:
    pred_file = os.path.join(DATA_DIR, exp, 'rot_mat_%s_%s_smap_%s_%s.txt' %(epoch, data_split, epoch_pose, exp_pose))
else:
    pred_file = os.path.join(DATA_DIR, exp, 'rot_mat_%s_%s.txt' %(epoch, data_split))
train_rot_mat_pred = pd.get_pred_rot_mat(pred_file, save_prefix)

keypoint_scale = pd.get_keypoint_scale(train_xyz_gt)

train_pred_xyz = []
for canon, rot, scale in zip(train_pred_xyz_canon, train_rot_mat_pred, keypoint_scale):
    train_pred_xyz.append(np.matmul(canon, rot)*scale)

train_pred_xyz += np.expand_dims(train_xyz_gt[:, 0, :], axis=1)
train_pred_xyz = np.asarray(train_pred_xyz).astype(np.float32)
train_pred_uvd = xyzuvd.xyz2uvd_color(np.asarray(train_pred_xyz))

In [ ]:
data_split = 'test'
save_prefix = 'test_fpha'
keys_cache_file = os.path.join(DATA_DIR, save_prefix + '_keys_cache.p')
test_keys = pickle.load(open(keys_cache_file, "rb"))

dataroot_xyz_gt_saved = os.path.join(DATA_DIR, save_prefix + '_xyz_gt.lmdb')

test_xyz_gt = pd.read_all_lmdb_from_file(test_keys, dataroot_xyz_gt_saved, np.float32, (21, 3))
test_xyz_gt = np.asarray(test_xyz_gt)
test_uvd_gt = xyzuvd.xyz2uvd_color(test_xyz_gt)

In [ ]:
dataroot_xyz_gt_canon = os.path.join(DATA_DIR, save_prefix + '_xyz_gt_canon.lmdb')
dataroot_rot_mat = os.path.join(DATA_DIR, save_prefix + '_rot_mat.lmdb')
test_xyz_gt_canon = pd.read_all_lmdb_from_file(test_keys, dataroot_xyz_gt_canon, np.float32, (21, 3))
test_rot_mat = pd.read_all_lmdb_from_file(test_keys, dataroot_rot_mat, np.float32, (3, 3))

if from_pred_smap:
    pred_file = os.path.join(DATA_DIR, exp, 'xyz_canon_%s_%s_smap_%s_%s.txt' %(epoch, data_split, epoch_pose, exp_pose))
else:
    pred_file = os.path.join(DATA_DIR, exp, 'xyz_canon_%s_%s.txt' %(epoch, data_split))
test_pred_xyz_canon = pd.get_pred_xyz_canon(pred_file, save_prefix)
if from_pred_smap:
    pred_file = os.path.join(DATA_DIR, exp, 'rot_mat_%s_%s_smap_%s_%s.txt' %(epoch, data_split, epoch_pose, exp_pose))
else:
    pred_file = os.path.join(DATA_DIR, exp, 'rot_mat_%s_%s.txt' %(epoch, data_split))
test_rot_mat_pred = pd.get_pred_rot_mat(pred_file, save_prefix)

keypoint_scale = pd.get_keypoint_scale(test_xyz_gt)

test_pred_xyz = []
for canon, rot, scale in zip(test_pred_xyz_canon, test_rot_mat_pred, keypoint_scale):
    test_pred_xyz.append(np.matmul(canon, rot)*scale)

test_pred_xyz += np.expand_dims(test_xyz_gt[:, 0, :], axis=1)
test_pred_xyz = np.asarray(test_pred_xyz).astype(np.float32)
test_pred_uvd = xyzuvd.xyz2uvd_color(np.asarray(test_pred_xyz))

In [ ]:

pred = train_xyz_gt_canon
true = train_pred_xyz_canon
print('TRAIN mean_l2_error CANON: ', error.mean_pose_error(true, pred))

pred = np.asarray(train_rot_mat_pred)
true = np.asarray(train_rot_mat)
errorrm = np.mean((true-pred)**2)
print('TRAIN mean_l2_error ROT MAT: ', errorrm)

pred = test_xyz_gt_canon
true = test_pred_xyz_canon
print('TEST mean_l2_error CANON: ', error.mean_pose_error(true, pred))

pred = np.asarray(test_rot_mat_pred)
true = np.asarray(test_rot_mat)
errorrm = np.mean((true-pred)**2)
print('TEST mean_l2_error ROT MAT: ', errorrm)


In [ ]:
coord_frame = 'uvd'
pred = train_pred_uvd
true = train_uvd_gt
print('%s TRAIN mean_l2_error: ' %coord_frame, error.mean_pose_error(true, pred))
pred = test_pred_uvd
true = test_uvd_gt
print('%s TEST mean_l2_error: ' %coord_frame, error.mean_pose_error(true, pred))

In [ ]:
coord_frame = 'xyz'
pred = train_pred_xyz
true = train_xyz_gt
print('%s TRAIN mean_l2_error: ' %coord_frame, error.mean_pose_error(true, pred))
pred = test_pred_xyz
true = test_xyz_gt
print('%s TEST mean_l2_error: ' %coord_frame, error.mean_pose_error(true, pred))

In [ ]:
# Percentage of correct key points
pred = train_pred_xyz
true = train_xyz_gt
pck = error.percentage_frames_within_error_curve_zimmmerman(true, pred)
print(pck)
thresholds = np.arange(0, 85, 5)
print('AUC:', error.calc_auc(pck, thresholds))

In [ ]:
# Percentage of correct key points
pred = test_pred_xyz
true = test_xyz_gt
pck = error.percentage_frames_within_error_curve_zimmmerman(true, pred)
print('[', end =" ")
for num in pck:
    print('%f, ' %num, end =" ")
print(']')
thresholds = np.arange(0, 85, 5)
print('AUC:', error.calc_auc(pck, thresholds))

In [ ]:
idx = 1659
file_name = os.path.join(DATASET_DIR, 'First_Person_Action_Benchmark', 'Video_files', test_keys[idx])
visual.show_true_pred_img_and_skel_color(file_name, test_uvd_gt[idx], test_pred_uvd[idx])

In [ ]:
file_name = os.path.join(DATASET_DIR, 'First_Person_Action_Benchmark', 'Video_files', train_keys[idx])
visual.show_true_pred_img_and_skel_color(file_name, train_uvd_gt[idx], train_pred_uvd[idx])